In [124]:
import sys
import requests
import pandas as pd 
import math

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    tp_data = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")

df = pd.read_json("database/refinement_recipes.json")
df = df.convert_dtypes()

output_item_price = []
crafting_profit = []
sell_velocity = []

ACQUISITION_TYPE = 'buy_price' # order buy
# ACQUISITION_TYPE = 'sell_price' # instant buy
# DISPOSAL_TYPE = 'buy_price' # instant sell
DISPOSAL_TYPE = 'sell_price' # order sell

for i, row in df.iterrows():

    tp_ingredient_cost = 0
    vendor_ingredient_cost = 0
    output_item_count = row['output_item_count']
    output_item_price.append(tp_data[tp_data['name'] == row['output_item_name']][DISPOSAL_TYPE].values[0])

    for ingredient in row['tp_ingredients']:
        ingredient['price'] = tp_data[tp_data['name'] == ingredient['ing_name']][ACQUISITION_TYPE].values[0]
        tp_ingredient_cost += ingredient['price'] * ingredient['count']

    for ingredient in row['vendor_ingredients']:
        vendor_ingredient_cost += ingredient['price'] * ingredient['count']

    # calculate refining profit
    crafting_price = (tp_ingredient_cost + vendor_ingredient_cost)/output_item_count
    sell_prize_minus_taxes = output_item_price[-1] - (output_item_price[-1] * 0.05) - (output_item_price[-1] * 0.1)
    crafting_profit.append(sell_prize_minus_taxes - crafting_price)

    # calculate selling daily velocity of 5% from last 7 days
    sell_velocity.append(int(tp_data[tp_data['name'] == row['output_item_name']]['7d_sell_sold'] / 7 * 0.05))


df['output_item_price'] = output_item_price
df['crafting_profit'] = crafting_profit
df['sell_velocity'] = sell_velocity


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[
        'output_item_name',
        'output_item_price',
        'crafting_profit',
        'sell_velocity',
        'tp_ingredients',
        'vendor_ingredients',
        'disciplines'
    ]]

	.sort_values(by='crafting_profit', ascending=False)

	.style.hide_index()
	)


# Chef: 441
# Armorsmith: 500
# Weaponsmith: 81
# Jeweler: 400


output_item_name,output_item_price,crafting_profit,sell_velocity,tp_ingredients,vendor_ingredients,disciplines
Cured Thick Leather Square,333.000000,47.050000,7485,"[{'ing_name': 'Thick Leather Section', 'count': 4, 'price': 59.0}]",[],"['Armorsmith', 'Artificer', 'Huntsman', 'Leatherworker', 'Scribe', 'Tailor', 'Weaponsmith']"
Hard Wood Plank,400.000000,40.000000,1337,"[{'ing_name': 'Hard Wood Log', 'count': 3, 'price': 100.0}]",[],"['Artificer', 'Huntsman', 'Scribe', 'Weaponsmith']"
Bolt of Wool,402.000000,29.700000,610,"[{'ing_name': 'Wool Scrap', 'count': 2, 'price': 156.0}]",[],"['Armorsmith', 'Leatherworker', 'Scribe', 'Tailor']"
Platinum Ingot,349.000000,24.650000,1430,"[{'ing_name': 'Platinum Ore', 'count': 2, 'price': 136.0}]",[],"['Scribe', 'Jeweler']"
Bronze Ingot,127.000000,20.350000,978,"[{'ing_name': 'Copper Ore', 'count': 10, 'price': 43.0}]","[{'ing_name': 'Lump of Tin', 'count': 1, 'price': 8}]","['Artificer', 'Weaponsmith', 'Scribe', 'Huntsman', 'Armorsmith']"
Cured Thin Leather Square,168.000000,18.800000,4580,"[{'ing_name': 'Thin Leather Section', 'count': 2, 'price': 62.0}]",[],"['Armorsmith', 'Artificer', 'Huntsman', 'Leatherworker', 'Scribe', 'Tailor', 'Weaponsmith']"
Bolt of Silk,57.000000,18.450000,6828,"[{'ing_name': 'Silk Scrap', 'count': 2, 'price': 15.0}]",[],"['Armorsmith', 'Leatherworker', 'Scribe', 'Tailor']"
Gold Ingot,104.000000,18.400000,424,"[{'ing_name': 'Gold Ore', 'count': 2, 'price': 35.0}]",[],"['Scribe', 'Jeweler']"
Darksteel Ingot,397.000000,17.450000,1263,"[{'ing_name': 'Platinum Ore', 'count': 2, 'price': 136.0}]","[{'ing_name': 'Lump of Primordium', 'count': 1, 'price': 48}]","['Scribe', 'Tailor', 'Artificer', 'Weaponsmith', 'Huntsman', 'Armorsmith', 'Leatherworker']"
Steel Ingot,456.000000,11.600000,3655,"[{'ing_name': 'Iron Ore', 'count': 3, 'price': 120.0}]","[{'ing_name': 'Lump of Coal', 'count': 1, 'price': 16}]","['Scribe', 'Tailor', 'Artificer', 'Weaponsmith', 'Huntsman', 'Armorsmith', 'Leatherworker']"


In [125]:

import pandas as pd 

df = pd.read_json("database/refinement_recipes.json")
df = df.convert_dtypes()

output_df = pd.DataFrame()

def get_crafting_profit(output_item_name, single_resource_price, refined_material_sell_price):
    global output_df

    # this value corresponds to cost of single resource needed to craft refined material
    tp_ingredient_cost = single_resource_price
    tp_ingredient_cost *= df[df['output_item_name'] == output_item_name]['tp_ingredients'].values[0][0]['count']

    vendor_ingredient_cost = 0
    for i in df[df['output_item_name'] == output_item_name]['vendor_ingredients'].values[0]:
        vendor_ingredient_cost += i['price'] * i['count']

    crafting_price = (tp_ingredient_cost + vendor_ingredient_cost)/df[df['output_item_name'] == output_item_name]['output_item_count'].values[0]
    sell_prize_minus_taxes = refined_material_sell_price - (refined_material_sell_price * 0.05) - (refined_material_sell_price * 0.1)
    profit = sell_prize_minus_taxes - crafting_price

    tmp_dict = {
        'output_item_name' : output_item_name,
        'single_resource_price': single_resource_price,
        'refined_material_sell_price': refined_material_sell_price,
        'refining_profit': profit,
        'crafting_price': crafting_price
    }

    output_df = output_df.append(tmp_dict, ignore_index=True)

get_crafting_profit('Steel Ingot', 129, 499)
get_crafting_profit('Platinum Ingot', 128, 0)

get_crafting_profit('Bolt of Silk', 17, 68)
get_crafting_profit('Bolt of Wool', 159, 402)

get_crafting_profit('Cured Thin Leather Square', 62, 170)
get_crafting_profit('Cured Thick Leather Square', 59, 0)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(output_df[[
        'output_item_name',
        'single_resource_price',
        'crafting_price',
        'refined_material_sell_price',
        'refining_profit'
    ]]

	# .sort_values(by='crafting_profit', ascending=False)

	# .style.hide_index()
	)

,output_item_name,single_resource_price,crafting_price,refined_material_sell_price,refining_profit
0,Steel Ingot,129.0,403.0,499.0,21.15
1,Platinum Ingot,128.0,256.0,0.0,-256.00
2,Bolt of Silk,17.0,34.0,68.0,23.80
3,Bolt of Wool,159.0,318.0,402.0,23.70
4,Cured Thin Leather Square,62.0,124.0,170.0,20.50
5,Cured Thick Leather Square,59.0,236.0,0.0,-236.00
